# CT as system of equations

In [ ]:
import sympy
import numpy as np
from tqdm import tqdm
import dill
dill.settings["recurse"] = True
from sympy import Rational as R

In [ ]:
(
    reference_x,
    reference_y,
    physical_x_0,
    physical_y_0,
    physical_x_1,
    physical_y_1,
    physical_x_2,
    physical_y_2,
) = sympy.symbols("x y x_0 y_0 x_1 y_1 x_2 y_2")

monomial_basis = sympy.Matrix(
[
 1,
 reference_x,
 reference_y,
 reference_x**2,
 reference_y**2,
 reference_x*reference_y,
 reference_x**3,
 reference_y**3,
 reference_x**2*reference_y,
 reference_x*reference_y**2,
 ]
)

rotation_matrix = sympy.Matrix([[+R(0), +R(1)], [-R(1), +R(0)]])


In [ ]:
v0 = {reference_x: 0, reference_y: 0}
v1 = {reference_x: 1, reference_y: 0}
v2 = {reference_x: 0, reference_y: 1}
barycenter = {reference_x: R(1, 3), reference_y: R(1, 3)}

e0 = {reference_x: R(1, 2), reference_y: R(1, 2)}
e1 = {reference_x: 0, reference_y: R(1, 2)}
e2 = {reference_x: R(1, 2), reference_y: 0}

e0_sub = {reference_x: R(1, 6), reference_y: R(1, 6)}
e1_sub = {reference_x: R(2, 3), reference_y: R(1, 6)}
e2_sub = {reference_x: R(1, 6), reference_y: R(2, 3)}

In [ ]:
t0_sub = sympy.Matrix([R(+1, 3), R(+1, 3)]) / sympy.sqrt(R(2, 9))
t1_sub = sympy.Matrix([R(-2, 3), R(+1, 3)]) / sympy.sqrt(R(5, 9))
t2_sub = sympy.Matrix([R(+1, 3), R(-2, 3)]) / sympy.sqrt(R(5, 9))

n0_sub = rotation_matrix @ t0_sub
n1_sub = rotation_matrix @ t1_sub
n2_sub = rotation_matrix @ t2_sub

In [ ]:
t0_hat = sympy.Matrix([-R(1), +R(1)]) / sympy.sqrt(2)
t1_hat = sympy.Matrix([+R(0), +R(1)])
t2_hat = sympy.Matrix([+R(1), +R(0)])

n0_hat = rotation_matrix @ t0_hat
n1_hat = rotation_matrix @ t1_hat
n2_hat = rotation_matrix @ t2_hat

In [ ]:
subtriangle_0_equation = (reference_y >= 0)&(reference_y <= reference_x)&(2*reference_y <= 1 - reference_x)
subtriangle_1_equation = (reference_y <= 1 - reference_x)&(reference_y >= -2*reference_x + 1)&(2*reference_y >= 1 - reference_x)
subtriangle_2_equation = (reference_x >= 0)&(reference_y <= -2*reference_x + 1)&(reference_y >= reference_x)

In [ ]:
coefs = sympy.Matrix([sympy.symbols("a_(0:3)_(0:10)")]).reshape(3, 10)

st_0_basis = (coefs.row(0)@monomial_basis)[0, 0]
st_1_basis = (coefs.row(1)@monomial_basis)[0, 0]
st_2_basis = (coefs.row(2)@monomial_basis)[0, 0]

eqn = []

fv0 = sympy.symbols("fv0")
dfdxv0 = sympy.symbols("dfdxv0")
dfdyv0 = sympy.symbols("dfdyv0")

fv1 = sympy.symbols("fv1")
dfdxv1 = sympy.symbols("dfdxv1")
dfdyv1 = sympy.symbols("dfdyv1")

fv2 = sympy.symbols("fv2")
dfdxv2 = sympy.symbols("dfdxv2")
dfdyv2 = sympy.symbols("dfdyv2")

dfdn0 = sympy.symbols("dfdn0")
dfdn1 = sympy.symbols("dfdn1")
dfdn2 = sympy.symbols("dfdn2")

# subtriangle 0
eqn.append(st_0_basis.subs(v0) - fv0)
eqn.append(st_0_basis.diff(reference_x).subs(v0) - dfdxv0)
eqn.append(st_0_basis.diff(reference_y).subs(v0) - dfdyv0)

eqn.append((n2_hat[0]*st_0_basis.diff(reference_x) + n2_hat[1]*st_0_basis.diff(reference_y)).subs(e2) - dfdn2)

# subtriangle 1
eqn.append(st_1_basis.subs(v1) - fv1)
eqn.append(st_1_basis.diff(reference_x).subs(v1) - dfdxv1)
eqn.append(st_1_basis.diff(reference_y).subs(v1) - dfdyv1)

eqn.append((n0_hat[0]*st_1_basis.diff(reference_x) + n0_hat[1]*st_1_basis.diff(reference_y)).subs(e0) - dfdn0)

# subtriangle 2
eqn.append(st_2_basis.subs(v2) - fv2)
eqn.append(st_2_basis.diff(reference_x).subs(v2) - dfdxv2)
eqn.append(st_2_basis.diff(reference_y).subs(v2) - dfdyv2)

eqn.append((n1_hat[0]*st_2_basis.diff(reference_x) + n1_hat[1]*st_2_basis.diff(reference_y)).subs(e1) - dfdn1)

# C1 continuity

# Global vertices
eqn.append(st_0_basis.subs(v1) - st_1_basis.subs(v1))
eqn.append(st_0_basis.diff(reference_x).subs(v1) - st_1_basis.diff(reference_x).subs(v1))
eqn.append(st_0_basis.diff(reference_y).subs(v1) - st_1_basis.diff(reference_y).subs(v1))

eqn.append(st_1_basis.subs(v2) - st_2_basis.subs(v2))
eqn.append(st_1_basis.diff(reference_x).subs(v2) - st_2_basis.diff(reference_x).subs(v2))
eqn.append(st_1_basis.diff(reference_y).subs(v2) - st_2_basis.diff(reference_y).subs(v2))

eqn.append(st_2_basis.subs(v0) - st_0_basis.subs(v0))
eqn.append(st_2_basis.diff(reference_x).subs(v0) - st_0_basis.diff(reference_x).subs(v0))
eqn.append(st_2_basis.diff(reference_y).subs(v0) - st_0_basis.diff(reference_y).subs(v0))

# Barycenter
eqn.append(st_1_basis.subs(barycenter) - st_0_basis.subs(barycenter))
eqn.append(st_1_basis.diff(reference_x).subs(barycenter) - st_0_basis.diff(reference_x).subs(barycenter))
eqn.append(st_1_basis.diff(reference_y).subs(barycenter) - st_0_basis.diff(reference_y).subs(barycenter))

eqn.append(st_2_basis.subs(barycenter) - st_0_basis.subs(barycenter))
eqn.append(st_2_basis.diff(reference_x).subs(barycenter) - st_0_basis.diff(reference_x).subs(barycenter))
eqn.append(st_2_basis.diff(reference_y).subs(barycenter) - st_0_basis.diff(reference_y).subs(barycenter))

# Subtriangle edges
eqn.append((n0_sub[0]*st_0_basis.diff(reference_x) + n0_sub[1]*st_0_basis.diff(reference_y)).subs(e0_sub) - 
           (n0_sub[0]*st_2_basis.diff(reference_x) + n0_sub[1]*st_2_basis.diff(reference_y)).subs(e0_sub)
           )

eqn.append((n1_sub[0]*st_1_basis.diff(reference_x) + n1_sub[1]*st_1_basis.diff(reference_y)).subs(e1_sub) - 
           (n1_sub[0]*st_0_basis.diff(reference_x) + n1_sub[1]*st_0_basis.diff(reference_y)).subs(e1_sub)
           )

eqn.append((n2_sub[0]*st_2_basis.diff(reference_x) + n2_sub[1]*st_2_basis.diff(reference_y)).subs(e2_sub) - 
           (n2_sub[0]*st_1_basis.diff(reference_x) + n2_sub[1]*st_1_basis.diff(reference_y)).subs(e2_sub)
           )


solved_coefs = sympy.solve(eqn)

solved_coefs = sympy.Matrix(list(solved_coefs.values())).reshape(3, 10)


subtriangle_0_basis = solved_coefs.row(0)@monomial_basis
subtriangle_1_basis = solved_coefs.row(1)@monomial_basis
subtriangle_2_basis = solved_coefs.row(2)@monomial_basis

solution_0 = sympy.Piecewise(
    (subtriangle_0_basis, subtriangle_0_equation),
)

solution_1 = sympy.Piecewise(
    (subtriangle_1_basis, subtriangle_1_equation),
)

solution_2 = sympy.Piecewise(
    (subtriangle_2_basis, subtriangle_2_equation),
)

# Map

In [ ]:
monomial_basis = sympy.Matrix(
    [
        1,
        reference_x,
        reference_y,
    ]
)

V = sympy.zeros(3, 3)

for dofidx, basis in enumerate(monomial_basis):

    V[dofidx, 0] = basis.subs({reference_x: 0, reference_y: 0})
    V[dofidx, 1] = basis.subs({reference_x: 1, reference_y: 0})
    V[dofidx, 2] = basis.subs({reference_x: 0, reference_y: 1})

mapping_basis = V.inv() @ monomial_basis

global_x = (
      mapping_basis[0] * physical_x_0
    + mapping_basis[1] * physical_x_1
    + mapping_basis[2] * physical_x_2
)
global_y = (
      mapping_basis[0] * physical_y_0
    + mapping_basis[1] * physical_y_1
    + mapping_basis[2] * physical_y_2
)

mapping_function = sympy.Matrix([global_x, global_y])
J = mapping_function.jacobian([reference_x, reference_y]).inv()

In [ ]:
omega = sympy.symbols("omega")

In [ ]:
# subtriangle_2_basis[0].subs({reference_x : omega, reference_y: omega}).diff(omega).subs({omega: R(1, 3)})

In [ ]:
grad = subtriangle_0_basis.jacobian([reference_x, reference_y])

In [ ]:
(J.T@grad.T).subs(barycenter)

In [ ]:
subtriangle_2_basis[0].diff(reference_x).subs(e0_sub)

## Coeff extraction

In [ ]:
coefficients = []
constants = []
for equation in eqn:
    coefficients.append([equation.expand().coeff(term) for term in sympy.symbols("a_(0:3)_(0:10)")])
    
# Create matrix and vector
A = sympy.Matrix(coefficients)

In [ ]:
poly_coefs = sympy.symbols("a_(0:6)")
omega = sympy.symbols("omega")

poly = sum([c*omega**n for n, c in enumerate(poly_coefs)])

eqn = [
        poly.diff(omega, 0).subs(omega, +1),
        poly.diff(omega, 0).subs(omega, -1),
        poly.diff(omega, 1).subs(omega, +1),
        poly.diff(omega, 1).subs(omega, -1),
        poly.diff(omega, 2).subs(omega, +1),
        poly.diff(omega, 2).subs(omega, -1),
    ]

b = poly.diff(omega, 1).subs(omega, 0)

coefficients = []
for equation in eqn:
    coefficients.append([equation.expand().coeff(term) for term in poly_coefs])
    
constants = [b.expand().coeff(term) for term in poly_coefs]
A = sympy.Matrix(coefficients)
b = sympy.Matrix(constants)

(A.inv().T@b).T


In [ ]:
poly_coefs = sympy.symbols("a_(0:4)")
omega = sympy.symbols("omega")

poly = sympy.Matrix([omega**n for n, c in enumerate(poly_coefs)])

N = len(poly)

Vander = sympy.Matrix(np.zeros((N, N)))

for i, p in enumerate(poly):
    
    Vander[i, 0] = p.diff(omega, 0).subs({omega: +1})
    Vander[i, 1] = p.diff(omega, 0).subs({omega: +0})
    
    Vander[i, 2] = p.diff(omega, 1).subs({omega: +1})
    Vander[i, 3] = p.diff(omega, 1).subs({omega: +0})
    
    # Vander[i, 4] = p.diff(omega, 2).subs({omega: +1})
    # Vander[i, 5] = p.diff(omega, 2).subs({omega: +0})    


In [ ]:
solution = Vander.inv()@poly

In [ ]:
poly_with_dofs = sympy.Matrix([poly_coefs])@solution

In [ ]:
L_2 = R(6)*omega**2 - R(6)*omega + R(1)
L_4 = R(70)*omega**4 - R(140)*omega**3 + R(90)*omega**2 - R(20)*omega + R(1)

In [ ]:
sympy.integrate(poly_with_dofs.diff(omega)*L_2, (omega, +0, +1))